## Сбор и подготовка данных.

In [1]:
# Импорты
import pandas as pd
import os
import torch
from transformers import GPT2Tokenizer
from transformers import pipeline
from rouge_score import rouge_scorer
from torch.utils.data import DataLoader
import sys
sys.path.append('src')
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
# Создание директорий
os.makedirs("data", exist_ok=True)
os.makedirs("models", exist_ok=True)

In [2]:
# Сбор и подготовка данных
from src.data_utils import *
download_and_extract_dataset()
prepare_dataset()
split_dataset()

Архив скачан: trainingdata.zip
Распаковано в папку temp_data
Файл перемещён: data/raw_dataset.csv
Временные файлы удалены
Очищенный датасет сохранён: data/dataset_processed.csv
train: 1277102, val: 159638, test: 159638


In [3]:
# DataLoader
from src.next_token_dataset import NextTokenDataset

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.save_pretrained("models/tokenizer/")

train_texts = pd.read_csv("data/train.csv")['text'].tolist()[:10000]
val_texts = pd.read_csv("data/val.csv")['text'].tolist()[:2000]

train_dataset = NextTokenDataset(train_texts, tokenizer, max_length=20)
val_dataset = NextTokenDataset(val_texts, tokenizer, max_length=20)

def collate_fn(batch, pad_token_id=50256):
    import torch
    from torch import nn

    input_ids = [torch.tensor(item['input_ids']) for item in batch]
    labels = [torch.tensor(item['labels']) for item in batch]

    # Паддинг до максимальной длины в батче
    input_ids = nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=pad_token_id)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=pad_token_id)

    return input_ids, labels

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)

## Реализация рекуррентной сети.

In [6]:
# Обучение LSTM
from lstm_train import train_lstm_model 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = train_lstm_model(train_loader, val_loader, vocab_size=tokenizer.vocab_size, device=device)

Epoch 1: 100%|███████████████████████████████████████████████████████████████████████| 156/156 [05:46<00:00,  2.22s/it]


Epoch 1, Loss: 7.9220


Epoch 2: 100%|███████████████████████████████████████████████████████████████████████| 156/156 [05:45<00:00,  2.22s/it]


Epoch 2, Loss: 7.1525


Epoch 3: 100%|███████████████████████████████████████████████████████████████████████| 156/156 [05:50<00:00,  2.25s/it]


Epoch 3, Loss: 7.0614


Epoch 4: 100%|███████████████████████████████████████████████████████████████████████| 156/156 [06:05<00:00,  2.34s/it]


Epoch 4, Loss: 7.0325


Epoch 5: 100%|███████████████████████████████████████████████████████████████████████| 156/156 [05:52<00:00,  2.26s/it]

Epoch 5, Loss: 6.9867
Модель сохранена: models/lstm_model.pth


## Оценка LSTM

In [11]:
# Оценка LSTM
sys.path.append('./src')
from eval_lstm import evaluate_lstm
lstm_rouge1, lstm_rouge2 = evaluate_lstm(model, val_loader, tokenizer, device=device)

Evaluating LSTM: 100%|█████████████████████████████████████████████████████████████████| 32/32 [12:51<00:00, 24.11s/it]

LSTM ROUGE-1: 0.3415
LSTM ROUGE-2: 0.2989


***Промежуточный вывод LSTM***

Несмотря на то, что LSTM и показала среднее качество ROUGE-1 и ROUGE-2, но модель повторяет «i», возможные причины:
 - Мало примеров т.е. данных, так как с целью экономии была ограничена выборка.
 - Возможно модель не научилась генерировать разнообразные слова.
 - Либо слишком мало эпох.
 - Может быть модель слишком простая.

## Использование предобученного трансформера

In [70]:
# Этап 4: Оценка DistilGPT-2
from eval_transformer_pipeline import evaluate_transformer
r1, r2 = evaluate_transformer(val_loader, tokenizer, device=device, max_examples=100)

DistilGPT-2: 100%|███████████████████████████████████████████████████████████████████| 100/100 [00:57<00:00,  1.74it/s]

DistilGPT-2 (на 100 примерах):
  ROUGE-1: 0.6468
  ROUGE-2: 0.5968


## Сравнение моделей

In [73]:
# Сравнение результатов LSTM и DistilGPT-2
# Загружаем модель DistilGPT-2
generator_DistilGPT = pipeline("text-generation", model="distilgpt2")

# Примеры промптов — начала фраз
examples = [
    "i love",
    "today is",
    "i feel",
    "this is",
    "i want"
]
print("Оценка автодополнения LSTM:")
for prompt in examples:
    generated = model.generate(tokenizer, prompt, max_length=20, device=device)
    print(f"Промпт: {prompt}")
    print(f"Дополнение LSTM: {generated}")
print("Оценка автодополнения DistilGPT:")
for prompt in examples:
    result = generator_DistilGPT(prompt, max_length=20, do_sample=True, top_k=50)
    generated = result[0]['generated_text']
    print(f"Промпт: {prompt}")
    print(f"Дополнение DistilGPT: {generated}")

Оценка автодополнения LSTM:
Промпт: i love
Дополнение LSTM: i love i i i i i i i i i i i i i i i i i i
Промпт: today is
Дополнение LSTM: today is i i i i i i i i i i i i i i i i i i
Промпт: i feel
Дополнение LSTM: i feel i i i i i i i i i i i i i i i i i i
Промпт: this is
Дополнение LSTM: this is i i i i i i i i i i i i i i i i i i
Промпт: i want
Дополнение LSTM: i want i i i i i i i i i i i i i i i i i i
Оценка автодополнения DistilGPT:
Промпт: i love
Дополнение DistilGPT: i love for these amazing images of her life, the story of her life as a singer and photographer
Промпт: today is
Дополнение DistilGPT: today is. When a new one comes out, the best information is always going to be out there
Промпт: i feel
Дополнение DistilGPT: i feel like it was just sitting on your couch (which made this sound quite annoying).
Промпт: this is
Дополнение DistilGPT: this is not used in the default case as a standard of functionality, it makes it very useful in
Промпт: i want
Дополнение DistilGPT: i w

***Вывод сравнение результатов LSTM и DistilGPT-2***

- LSTM: ROUGE-1 ~0.34, ROUGE-2 ~0.29, Низкий качество и уровень: повторяет слова, не строит осмысленные фразы.
- DistilGPT-2: ROUGE-1 ~0.65, ROUGE-2 ~0.60, Высокий уровень: генерирует логичные, связные и естественные предложения.